In [1]:
from crypto_env import Market
import pandas as pd
import numpy as np
from utils import *
from ac import Actor, Critic
from dqn import ReplayMemory, DQN_Agent
import gym
import torch
import gym_anytrading

from stable_baselines3 import DQN, A2C
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
df = pd.read_csv('data/raw/FTX/btc_usd_1h.csv', header=None)
df.columns = ['ts', 'open', 'high', 'low', 'close', 'volume']

In [3]:
market_data = df.copy()
market_data = market_data.drop("ts", axis=1)

In [4]:
market_train = market_data[:int(len(market_data)/2)]
market_test = market_data[int(len(market_data)/2):]

### DQN on Market Environment

In [ ]:
env = Market(market_train, initial_balance=1000000)

agent = DQN_Agent(memory_length=500, state_dim=len(env.observation_space.sample()), num_actions=3)

train_dqn(agent, env, batch_size=32, discount=0.99, eps_init=1, eps_decay=0.99, eps_min=0.1, num_actions=3, n_episodes=100)


Episode:  0
Price: 10756.0, Total Reward: 58101552.894238904, Account Value: 937943.9653456229

Episode:  1
Price: 10756.0, Total Reward: 60934636.14177145, Account Value: 1030571.6173972158

Episode:  2
Price: 10756.0, Total Reward: 18805031.100533407, Account Value: 601885.5134717441

Episode:  3
Price: 10756.0, Total Reward: 231010.64181555645, Account Value: 1231010.6418155564

Episode:  4
Price: 10756.0, Total Reward: 2061469.9984156904, Account Value: 1143176.3139496509

Episode:  5
Price: 10756.0, Total Reward: 177750.86402666057, Account Value: 1177750.8640266606

Episode:  6
Price: 10756.0, Total Reward: 1505573.3241936243, Account Value: 541970.939825792

Episode:  7
Price: 10756.0, Total Reward: 3730373.4345033504, Account Value: 1431987.5971620157

Episode:  8
Price: 10756.0, Total Reward: 3293717.805139066, Account Value: 1140206.7079408846

Episode:  9
Price: 10756.0, Total Reward: 7172600.1414088, Account Value: 1081318.0969862556

Episode:  10
Price: 10756.0, Total Rew

In [ ]:
market_train.head(10)

### Stable-Baselines DQN on Market Environment

In [ ]:
gym.register(
    id='Market-v0',
    entry_point='crypto_env:Market'
)

env_maker = lambda: gym.make(
    'Market-v0',
    market_data = market_train,
    initial_balance = 1000000
)

env = DummyVecEnv([env_maker])

In [ ]:
agent = DQN(policy='MlpPolicy', env=env)
trained_dqn = agent.learn(total_timesteps=100000)

In [ ]:
test_env_maker = lambda: gym.make('Market-v0', market_data=market_test.reset_index().drop('index',axis=1), initial_balance=1000000)
test_env = test_env_maker()

In [ ]:
with torch.no_grad():
    obs = test_env.reset()
    
    s = obs
    while s is not None:
        a, _ = trained_dqn.predict(torch.FloatTensor(s))
        s_next, r, done, info = test_env.step(a)
        
        if done:
            s = None
        else:
            s = s_next
test_env.close()

In [ ]:
test_env.render()

### Stable-Baselines A2C on Market Environment

In [ ]:
env_maker = lambda: gym.make(
    'Market-v0',
    market_data = market_train,
    initial_balance = 1000000
)

env = DummyVecEnv([env_maker])

In [ ]:
agent = A2C(policy='MlpPolicy', env=env)
trained_a2c = agent.learn(total_timesteps=100000)

In [ ]:
test_env_maker = lambda: gym.make('Market-v0', market_data=market_test.reset_index().drop('index',axis=1), initial_balance=1000000)
test_env = test_env_maker()

In [ ]:
with torch.no_grad():
    obs = test_env.reset()
    
    s = obs
    while s is not None:
        a, _ = trained_dqn.predict(torch.FloatTensor(s))
        s_next, r, done, info = test_env.step(a)
        
        if done:
            s = None
        else:
            s = s_next
# test_env.close()

In [ ]:
test_env.render()

In [ ]:
test_env.close()

In [ ]:
train_ac(actor, critic, env, 0.99)